In [1]:
import os
import glob
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt

## extract irregular boundary

In [2]:
# this bound is irregular
bound = xr.open_rasterio('D:/ERA5/extracted.tif')
bound1 = bound.to_dataset(name='pre')
# lon should be sorted in ascending order while lat in descending order in north hemisphere
lon = bound1.x.values
lat = bound1.y.values
lon1 = sorted(lon, reverse=True)
lat1 = sorted(lat, reverse=True)

C:\Users\bn_li\AppData\Local\Temp\ipykernel_516\3167017885.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  bound = xr.open_rasterio('D:/ERA5/extracted.tif')


In [3]:
bound1

<xarray.Dataset>
Dimensions:  (band: 1, y: 39, x: 47)
Coordinates:
  * band     (band) int32 1
  * y        (y) float64 34.0 33.75 33.5 33.25 33.0 ... 25.25 25.0 24.75 24.5
  * x        (x) float64 110.5 110.8 111.0 111.2 ... 121.2 121.5 121.8 122.0
Data variables:
    pre      (band, y, x) float64 ...

In [4]:
row = bound1.y.size
col = bound1.x.size

In [5]:
wlat = []
wlon = []


for i in lat:
    found = False
    for j in lon:
        if not np.isnan(bound1.sel(x=j,y=i)['pre'].values):
            wlat.append(i)
            wlon.append(j)
            found = True
            break
        if found:
            continue

wlonlat = {'wlon':wlon,'wlat':wlat}
df = pd.DataFrame(wlonlat)
df.to_csv('D:/ERA5/wlonlat.csv')

elat = []
elon = []


for i in lat:
    found = False
    for j in lon1:
        if not np.isnan(bound1.sel(x=j,y=i)['pre'].values):
            elat.append(i)
            elon.append(j)
            found = True
            break
        if found:
            continue

elonlat = {'elon':elon,'elat':elat}
df = pd.DataFrame(elonlat)
df.to_csv('D:/ERA5/elonlat.csv')

slat = []
slon = []


for j in lon:
    found = False
    for i in lat:
        if not np.isnan(bound1.sel(x=j,y=i)['pre'].values):
            slat.append(i)
            slon.append(j)
            found = True
            break
        if found:
            continue

slonlat = {'slon':slon,'slat':slat}
df = pd.DataFrame(slonlat)
df.to_csv('D:/ERA5/slonlat.csv')

nlat = []
nlon = []


for j in lon:
    found = False
    for i in lat1:
        if not np.isnan(bound1.sel(x=j,y=i)['pre'].values):
            nlat.append(i)
            nlon.append(j)
            found = True
            break
        if found:
            continue

nlonlat = {'nlon':nlon,'nlat':nlat}
df = pd.DataFrame(nlonlat)
df.to_csv('D:/ERA5/nlonlat.csv')

## cal flux

In [2]:
ds= xr.open_mfdataset('D:/ERA5/WV0/*.nc')
dsy = ds.resample(time='M').mean(skipna=False)
dsy.to_netcdf('D:/ERA5/ERA5_WV_M0.nc')

In [13]:
dsm = xr.open_dataset('D:/ERA5/ERA5_WV_M.nc')
dsm

<xarray.Dataset>
Dimensions:    (longitude: 129, latitude: 51, time: 264)
Coordinates:
  * longitude  (longitude) float32 90.0 90.25 90.5 90.75 ... 121.5 121.8 122.0
  * latitude   (latitude) float32 36.5 36.25 36.0 35.75 ... 24.5 24.25 24.0
  * time       (time) datetime64[ns] 2001-01-31 2001-02-28 ... 2022-12-31
Data variables:
    p71.162    (time, latitude, longitude) float32 ...
    p72.162    (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-01 03:36:26 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [14]:
mon = dsm.time.size

In [15]:
dfwm = pd.read_csv('D:/ERA5/wlonlat.csv')
dfem = pd.read_csv('D:/ERA5/elonlat.csv')
dfnm = pd.read_csv('D:/ERA5/nlonlat.csv')
dfsm = pd.read_csv('D:/ERA5/slonlat.csv')

In [16]:
westf = np.zeros((row,mon))
for time in range(0,mon):
    for i in range(0,row): # 47 rows after extract
        westf[i,time] = dsm['p71.162'].sel(latitude=dfwm['wlat'][i],
                                   longitude=dfwm['wlon'][i],
                                  time = dsm.time[time]).values
        

eastf = np.zeros((row,mon))
for time in range(0,mon):
    for i in range(0,row): # 46 rows after extract
        eastf[i,time] = dsm['p71.162'].sel(latitude=dfem['elat'][i],
                                   longitude=dfem['elon'][i],
                                  time = dsm.time[time]).values

southf = np.zeros((mon,col))
for time in range(0,mon):
    for i in range(0,col): # 46 rows after extract
        southf[time,i] = dsm['p72.162'].sel(latitude=dfsm['slat'][i],
                                   longitude=dfsm['slon'][i],
                                  time = dsm.time[time]).values
        
northf = np.zeros((mon,col))
for time in range(0,mon):
    for i in range(0,col): # 46 rows after extract
        northf[time,i] = dsm['p72.162'].sel(latitude=dfnm['nlat'][i],
                                   longitude=dfnm['nlon'][i],
                                  time = dsm.time[time]).values

In [17]:
# monthly flux

flux =np.zeros((mon,2))
for time in range(0,mon):
    win = np.array(0)
    wout = np.array(0)
    eout = np.array(0)
    ein = np.array(0)
    sin = np.array(0)
    sout = np.array(0)
    nout = np.array(0)
    nin = np.array(0)

    


    for i in range(0,row): # 46 rows after extract
        if westf[i,time] >= 0:
            win = westf[i,time]+win
        else:
            wout = westf[i,time]+wout
        if eastf[i,time] >= 0:
            eout = eastf[i,time]+eout
        else:
            ein = eastf[i,time]+ein
        
        
        
    for j in range(0,col):
        if southf[time,j] >= 0:
            sin = southf[time,j]+sin
        else:
            sout = southf[time,j]+sout
        if northf[time,j] >= 0:
            nout = northf[time,j]+nout
        else:
            nin = northf[time,j]+nin
    

    fxin = (win+ein)*27750*86400*30 #kg
    fxout = (eout+wout)*27750*86400*30 #kg
    fyin = (sin+nin)*27750*86400*30 #kg
    fyout = (sout+nout)*27750*86400*30 #kg
    fin = np.sqrt(fxin**2+fyin**2)
    fout = np.sqrt(fxout**2+fyout**2)
    flux[time,0] = fin
    flux[time,1] = fout


In [19]:
# JAS flux

flux =np.zeros((mon,2))

for time in range(6,mon,12):
    win = np.array(0)
    wout = np.array(0)
    eout = np.array(0)
    ein = np.array(0)
    sin = np.array(0)
    sout = np.array(0)
    nout = np.array(0)
    nin = np.array(0)

    


    for i in range(0,row): # 46 rows after extract
        if ((westf[i,time]+westf[i,time+1]+westf[i,time+2])/3) >= 0:
            win = (westf[i,time]+westf[i,time+1]+westf[i,time+2])/3+win
        else:
            wout = (westf[i,time]+westf[i,time+1]+westf[i,time+2])/3+wout
                   
        if ((eastf[i,time]+eastf[i,time+1]+eastf[i,time+2])/3) >= 0:
            eout = (eastf[i,time]+eastf[i,time+1]+eastf[i,time+2])/3+eout
        else:
            ein = (eastf[i,time]+eastf[i,time+1]+eastf[i,time+2])/3+ein
        
        
        
    for j in range(0,col):
        if ((southf[time,j]+southf[time+1,j]+southf[time+2,j])/3) >= 0:
            sin = (southf[time,j]+southf[time+1,j]+southf[time+2,j])/3+sin
        else:
            sout = (southf[time,j]+southf[time+1,j]+southf[time+2,j])/3+sout
            
        if ((northf[time,j]+northf[time+1,j]+northf[time+2,j])/3) >= 0:
            nout = (northf[time,j]+northf[time+1,j]+northf[time+2,j])/3+nout
        else:
            nin = (northf[time,j]+northf[time+1,j]+northf[time+2,j])/3+nin
    

    fxin = (win+ein)*27750*86400*30*3 #kg
    fxout = (eout+wout)*27750*86400*30*3 #kg
    fyin = (sin+nin)*27750*86400*30*3 #kg
    fyout = (sout+nout)*27750*86400*30*3 #kg
    fin = np.sqrt(fxin**2+fyin**2)
    fout = np.sqrt(fxout**2+fyout**2)
    print(fout)

625666084229752.5
254584466292734.47
1236059209284768.2
906318285042718.2
1068045937299546.1
638073511690082.1
858597103998557.9
851151133208229.8
577090401504578.2
1088863148222375.9
818630943109827.6
632519474321810.9
1368125721773395.2
951473352736663.6
487104788044441.06
537371481115403.4
1287324804044776.5
767833975432636.1
311490572739371.06
1408824604346466.8
849913358723156.4
739956465524252.8


## cal et

In [14]:
ds= xr.open_mfdataset('D:/ERA5/ET1/*.nc')
dsm = ds.resample(time='M').sum(skipna=False)
dsm.to_netcdf('D:/ERA5/ET_M1.nc')

In [33]:
dsm1 = xr.open_dataset('D:/ERA5/ERA5_ET_M.nc')
mons = dsm1.time.size

In [29]:
bound = xr.open_rasterio('D:/ERA5/extracted.tif')
bound1 = bound.to_dataset(name='e')
bound1 = bound1.rename({'x':'longitude','y':'latitude','band':'time'})
bound1

C:\Users\bn_li\AppData\Local\Temp\ipykernel_516\3797961515.py:1: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  bound = xr.open_rasterio('D:/ERA5/extracted.tif')


<xarray.Dataset>
Dimensions:    (time: 1, latitude: 39, longitude: 47)
Coordinates:
  * time       (time) int32 1
  * latitude   (latitude) float64 34.0 33.75 33.5 33.25 ... 25.0 24.75 24.5
  * longitude  (longitude) float64 110.5 110.8 111.0 111.2 ... 121.5 121.8 122.0
Data variables:
    e          (time, latitude, longitude) float64 ...

In [34]:
dsm2 = dsm1.sel(latitude=bound1.latitude.values,longitude=bound1.longitude.values,time=dsm1.time.values)
dsm3 = dsm2.sum(dim=['latitude','longitude']) 
et = (dsm3['e'].values)*(-1)*27750*27750*1000

for time in range(6,mons,12):
    et3 = np.array(0)
    et3 = et3+et[time]+et[time+1]+et[time+2]
    print(et3)

480470550708224.0
469436251242496.0
445350351470592.0
478680631476224.0
445815793385472.0
472781376454656.0
454511382495232.0
466493863100416.0
448687155183616.0
444002436382720.0
442557129555968.0
477241767100416.0
463813518295040.0
429079547346944.0
450328386338816.0
466927528968192.0
459076370694144.0
476470543646720.0
466531360178176.0
434864297869312.0
455678103650304.0
446371698049024.0


## cal PRR